# Welcome to TQ42

## Example of training a generic time series Neural Network

In [15]:
#Import Packages
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto

from tq42.algorithm import (
    TrainDataProcessingParametersProto,
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto, 
    ActFuncProto,
    GenericMLTrainMetadataProto,
    GenericMLTrainParametersProto,
    Layer,
    PQNLayer,
    ClassicalDenseLayer,
    MLModelType,
    ActivationFunctionLayer,
    ActFuncProto,
    DropoutLayer,
    BatchNormalizationLayer,
    TrainModelInfoProto,
    MLTrainInputsProto,
    AlgorithmProto,
    MeasurementModeProto,
    MeasureProto,
    EntanglingProto,
    DiffMethodProto,
    QubitTypeProto,
    ClassicalLSTMLayer,
    QLSTMLayer,
)

from google.protobuf.json_format import MessageToDict

# Create a client. Select an Organization, Project and Experiment

In [2]:
# Connect to the cloud service
# https://terraquantum.io
with TQ42Client() as client:
    client.login()

{"device_code":"oTm2rAZRTRTLtfWWOjco6qD9","user_code":"FPDR-MMQV","verification_uri":"https://auth.terraquantum.io/activate","expires_in":900,"interval":5,"verification_uri_complete":"https://auth.terraquantum.io/activate?user_code=FPDR-MMQV"}
If a browser does not open, please access this URL: https://auth.terraquantum.io/activate?user_code=FPDR-MMQV to login. Also check the code: FPDR-MMQV
Authentication is successful, access token is saved in: keyring.
org:2b040bcb-1697-491f-83d8-9da045c93eaf
proj:347826c3-5b62-4bd7-ad06-cfd11b9e4746


In [3]:
# list the organizations available to you 
for organization in list_all_organizations(client=client):
    print (organization.id)

2b040bcb-1697-491f-83d8-9da045c93eaf


In [4]:
# Select an organization from the list
org_selected = list_all_organizations(client=client)[0]
print (org_selected.id)

2b040bcb-1697-491f-83d8-9da045c93eaf


In [5]:
# List the projects within that organization
for project in list_all_projects(client=client, organization_id=org_selected.id):
    print (project.id)

347826c3-5b62-4bd7-ad06-cfd11b9e4746
3973990a-3f89-4096-8491-0f2866e31344


In [7]:
# Select a project from the list
proj_selected = list_all_projects(client=client, organization_id=org_selected.id)[0]
print(proj_selected.id)

347826c3-5b62-4bd7-ad06-cfd11b9e4746


In [8]:
# List the experiments within that project
for experiment in list_all_experiments(client=client, project_id=proj_selected.id):
    print (experiment.id)

e8ca73ae-d9a2-41a2-aeb8-7a734916fd72
5921506d-b7aa-4aa9-a934-1d427a5c9a93
75b3aaaa-405c-4312-8ea5-a613460d8cea
0762f32e-b2ce-4e58-953c-0a6741e25266
91918c0c-7e2c-409b-8b7b-0409d7c36846
0ba18e6f-65e6-4c0a-bda3-091c5a45312d
3f4fcf32-bc79-4147-971b-ed56b2cc8cd5


In [9]:
# Select an experiment from the list
exp_selected = list_all_experiments(client=client, project_id=proj_selected.id)[0]
print(exp_selected.id)

e8ca73ae-d9a2-41a2-aeb8-7a734916fd72


# Build and Train the Neural Network

### Classical Example

In [16]:
# Build the classical neural network

classical_train_message=MessageToDict(GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        # Choose model type here
        model_type=MLModelType.MLP,
        # Add and customize and customize layers here
        layers=[
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=30, bias=True)),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=45, bias=True)),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=60, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(dropout_layer=DropoutLayer(value=0.5)),
            Layer(batch_normalization_layer=BatchNormalizationLayer()),
            Layer(
                classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=False)
            ),  # Final layer, bias=False
        ],
        num_epochs=5,
        batch_size=128,
        learning_rate=0.05,
        k_fold=1,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model.
            name="ENTER_MODEL_NAME_HERE",
            # Add a brief description to help users understand the purpose or functionality of this trained model.
            description="ADD_DESCRIPTION_HERE",
        ),
        data_processing_parameters=TrainDataProcessingParametersProto(
            input_columns=[0, 1, 2, 3],
            output_columns=[4]
        )
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id="1023200d-6d36-45ea-b66d-afbcc192eabc")
    ),
),preserving_proto_field_name=True)

In [17]:
# Train the classical neural network

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN, 
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL, 
    parameters=classical_train_message
)

print(run.data)

id: "9140b1ce-17ef-4506-9123-1d69c0d60e81"
experiment_id: "e8ca73ae-d9a2-41a2-aeb8-7a734916fd72"
sequential_id: 4
status: QUEUED
algorithm: GENERIC_ML_TRAIN
hardware: SMALL
metadata: "{\"parameters\":{\"model_type\":\"MLP\",\"layers\":[{\"classical_dense_layer\":{\"hidden_size\":30,\"bias\":true}},{\"classical_dense_layer\":{\"hidden_size\":45,\"bias\":true}},{\"classical_dense_layer\":{\"hidden_size\":60,\"bias\":true}},{\"activation_function_layer\":{\"function\":\"SIGMOID\"}},{\"dropout_layer\":{\"value\":0.5}},{\"batch_normalization_layer\":{}},{\"classical_dense_layer\":{\"hidden_size\":1}}],\"num_epochs\":5,\"batch_size\":128,\"learning_rate\":0.05,\"optim\":\"ADAM\",\"loss_func\":\"MSE\",\"train_model_info\":{\"name\":\"ENTER_MODEL_NAME_HERE\",\"description\":\"ADD_DESCRIPTION_HERE\"},\"data_processing_parameters\":{\"input_columns\":[0,1,2,3],\"output_columns\":[4]},\"k_fold\":1},\"inputs\":{\"data\":{\"storage_id\":\"1023200d-6d36-45ea-b66d-afbcc192eabc\"}}}"
result {
}
create

### Hybrid Quantum Example

In [ ]:
# Build the hybrid quantum neural network

hybrid_quantum_train_message=MessageToDict(GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        # Choose model type here
        model_type=MLModelType.MLP,
        # Add and customize and customize layers here
        layers=[
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=4, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=4, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(
                pqn_layer=PQNLayer(
                    num_qubits=4,
                    depth=7,
                    measurement_mode=MeasurementModeProto.NONE,
                    rotation=MeasureProto.X,
                    entangling=EntanglingProto.BASIC,
                    measure=MeasureProto.Z,
                    diff_method=DiffMethodProto.ADJOINT,
                    qubit_type=QubitTypeProto.LIGHTNING_QUBIT,
                )
            ),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=False)),
        ],
        num_epochs=5,
        k_fold=1,
        batch_size=128,
        learning_rate=0.05,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model.
            name="ENTER_MODEL_NAME_HERE",
            # Add a brief description to help users understand the purpose or functionality of this trained model.
            description="ADD_DESCRIPTION_HERE",
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id="ENTER_DATASET_STORAGE_ID_HERE")
    ),
),preserving_proto_field_name=True)

In [ ]:
# Train the hybrid quantum neural network

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN, 
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL, 
    parameters=hybrid_quantum_train_message
)

print(run.data)